In [1]:
import cadquery as cq
import numpy as np
from cadquery import selectors
from cadquery import exporters
from jupyter_cadquery import set_defaults
from jupyter_cadquery.replay import enable_replay
from jupyter_cadquery.viewer.client import show, show_object
from cqkit import EdgeLengthSelector #separate library that is helpful

import random

enable_replay(False)

set_defaults(axes=False, axes0=False)
set_defaults(theme="dark")

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [4]:
LOCAL_PATH = '' #Path to CAD data

nozzle = (
    cq.importers.importStep(LOCAL_PATH)
)

nozzle

AttributeError: module 'cadquery.occ_impl.importers' has no attribute 'importSTL'

In [8]:
external_channel_gap = 1.75
external_channel_start = 15.5
internal_channel_end = 32.36

def internal_channel_radius(x):
    x = abs(x)
    if x < 8:
        r = 8
    elif x < 12:
        r = 4 + (12 - x)
    elif x < 15.73:
        r = 4
    elif x < 19.63:
        r = 1.75 + ((19.63 - x) * (2.25 / 3.9))
    elif x < 27.63:
        r = 1.75
    elif x < 29.36:
        r = 0.75 + ((29.36 - x) * (1 / 1.73))
    elif x < 32.36:
        r = 0.75

    return r

def external_channel_radius(x):
    x = abs(x)
    if x < 18.3:
        r = 8
    elif x < 23.5:
        r = 5 + ((23.5 - x) * (3 / 5.2))
    elif x < 28.5:
        r = 5
    elif x < 33.5:
        r = 2.11 + ((33.5 - x) * (2.89 / 5))
    elif x <= 35:
        r = 2.11

    return r

In [6]:
np.array(range(3,4))

array([3])

In [13]:
for defect_morphology in ['clog']:#, 'void']:
    for defect_type in ['blob']:#['sphere', 'cylinder', 'rect', 'cone', 'spikey', 'blob']:#['blob', 'spikey']:
        for id in range(3, 4):
                nozzle = (
                    cq.importers.importStep(LOCAL_PATH)
                )

                def generate_sphere(size):
                    blob = (cq.Workplane()
                        .sphere(random.random()*size/2).translate(((random.random() - 1)*size, (random.random() - 1)*size, (random.random() - 1)*size))
                        )
                    return blob

                def generate_blob(size, blob_num):
                    blob = generate_sphere(size)
                    for idx in range(blob_num):
                        blob = blob.union(generate_sphere(size))
                    return blob
            

                # defect_radius = 1/2
                #defect_radius = 2/2
                # defect_radius = 2

                defect_radius_array = np.array([.5, .5, .5, .5, 1, 1, 1, 1, 2, 2, 2, 2])
                defect_spacing = defect_radius_array*2 + .5

                defect_order = np.random.permutation(len(defect_radius_array))
                defect_start = np.random.uniform(0.09, 1 - 0.09)

                defect_x_loc = [defect_spacing[defect_order[0]]/2 + defect_start]

                for idx in range(len(defect_radius_array) - 1):
                    defect_x_loc = np.append(defect_x_loc, defect_x_loc[idx] + defect_spacing[defect_order[idx]]/2 + defect_spacing[defect_order[idx + 1]]/2)

                
                if defect_type == 'blob':
                        blob_1 = generate_blob(.5, 25)
                        blob_2 = generate_blob(1, 25)
                        blob_4 = generate_blob(2, 25)

                    

                for idx in range(len(defect_radius_array)):
                    
                    x = defect_x_loc[idx]
                    defect_radius = defect_radius_array[defect_order[idx]]

                    if defect_type == 'blob':
                        if defect_radius == .5:
                            blob = blob_1
                        elif defect_radius == 1:
                            blob = blob_2
                        elif defect_radius == 2:
                            blob = blob_4

                    if defect_type == 'spikey':
                        spikey = (
                            cq.importers.importStep('C:/Users/bimrose2/Box/ZJU UIUC - CT Defect Detection/Nozzle CAD/spikey_'  + str(int(defect_radius*2)) + 'mm.STEP')
                        )

                    external = False

                    if x < external_channel_start:
                        r = internal_channel_radius(x)

                    elif x > internal_channel_end:
                        r = external_channel_radius(x)
                    else:
                        external = random.choice([True, False])
                        x_choices = [internal_channel_radius(x), external_channel_radius(x), ]
                        r = x_choices[int(external)]

                    defect_theta = random.random() * 2 * np.pi

                    if external & (x < 33.5):
                        r = random.choice([r - (1 - random.random()) * (defect_radius), (r - 1.75) - (1 - random.random()) * (defect_radius)])
                    else:
                        r = r + (1 - random.random()) * (defect_radius * 2/3)

                    defect_y = r * np.cos(defect_theta)
                    defect_z = r * np.sin(defect_theta)

                    # Spherical Defect
                    if defect_type == 'sphere':
                        if defect_morphology == 'void':
                            nozzle = nozzle.cut(cq.Workplane("YZ").sphere(defect_radius).translate((-x, defect_y, defect_z)))
                        else:
                            nozzle = nozzle.union(cq.Workplane("YZ").sphere(defect_radius).translate((-x, defect_y, defect_z)))
                    # Cylinderical Defect
                    if defect_type == 'cylinder':
                        if defect_morphology == 'void':
                            nozzle = nozzle.cut(cq.Workplane("YZ").cylinder(defect_radius*1.5, defect_radius*.5).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
                        else:
                            nozzle = nozzle.union(cq.Workplane("YZ").cylinder(defect_radius*1.5, defect_radius*.5).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
                    # Rectangular Defect
                    if defect_type == 'rect':
                        if defect_morphology == 'void':
                            nozzle = nozzle.cut(cq.Workplane("YZ").rect(defect_radius*1.5, defect_radius*.5).extrude(random.random()*1.5*defect_radius).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
                        else:
                            nozzle = nozzle.union(cq.Workplane("YZ").rect(defect_radius*1.5, defect_radius*.5).extrude(random.random()*1.5*defect_radius).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
                    # Cone Defect
                    if defect_type == 'cone':
                        if defect_morphology == 'void':
                            nozzle = nozzle.cut(cq.Workplane("YZ").circle(defect_radius/2).workplane(offset=defect_radius*1.5).circle(0.01).loft(combine=True).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
                        else:
                            nozzle = nozzle.union(cq.Workplane("YZ").circle(defect_radius/2).workplane(offset=defect_radius*1.5).circle(0.01).loft(combine=True).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
                    # spikey Defect
                    if defect_type == 'spikey':
                        if defect_morphology == 'void':
                            nozzle = nozzle.cut(spikey.translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
                        else:
                            nozzle = nozzle.union(spikey.translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
                    # blob Defect:
                    if defect_type == 'blob':
                        if defect_morphology == 'void':
                            nozzle = nozzle.cut(blob.translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
                        else:
                            nozzle = nozzle.union(blob.translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))

                #exporters.export(nozzle, 'C:/Users/bimrose2/Box/ZJU UIUC - CT Defect Detection/Nozzle CAD/Synthetic_Defects/nozzle_r1_clog_' + str(id) + '.stl')
                nozzle.val().exportStl('cad/' + str(defect_type) + '_' + str(defect_morphology) + '_' + str(id) + '.stl',tolerance=0.001,angularTolerance=.2, ascii=False)

nozzle

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='dark', title=None, …

In [111]:
defect_morphology = ['clog', 'void']
defect_type = ['sphere', 'cylinder', 'rect', 'cone', 'spikey', 'blob']


defect_morphology = 'clog'
defect_type = 'blob'

nozzle = (
    cq.importers.importStep(LOCAL_PATH)
)

def generate_sphere(size):
    blob = (cq.Workplane()
        .sphere(random.random()*size/2).translate(((random.random() - 1)*size, (random.random() - 1)*size, (random.random() - 1)*size))
        )
    return blob

def generate_blob(size, blob_num):
    blob = generate_sphere(size)
    for idx in range(blob_num):
        blob = blob.union(generate_sphere(size))
    return blob


defect_radius = 1/2
#defect_radius = 2/2
# defect_radius = 2

defect_radius_array = np.array([.5, .5, .5, .5, 1, 1, 1, 1, 2, 2, 2, 2])
defect_spacing = defect_radius_array*2 + .5

defect_order = np.random.permutation(len(defect_radius_array))
defect_start = np.random.uniform(0.09, 1 - 0.09)

defect_x_loc = [defect_spacing[defect_order[0]]/2 + defect_start]

for idx in range(len(defect_radius_array) - 1):
    defect_x_loc = np.append(defect_x_loc, defect_x_loc[idx] + defect_spacing[defect_order[idx]]/2 + defect_spacing[defect_order[idx + 1]]/2)


if defect_type == 'blob':
        blob_1 = generate_blob(.5, 25)
        blob_2 = generate_blob(1, 25)
        blob_4 = generate_blob(2, 25)

    
    
x = np.random.uniform(8 + 2.09, 35-2.09)
#defect_radius = defect_radius_array[defect_order[idx]]

if defect_type == 'blob':
    if defect_radius == .5:
        blob = blob_1
    elif defect_radius == 1:
        blob = blob_2
    elif defect_radius == 2:
        blob = blob_4

if defect_type == 'spikey':
    spikey = (
        cq.importers.importStep('cad/spikey_'  + str(int(defect_radius*2)) + 'mm.STEP')
    )

external = False

if x < external_channel_start:
    r = internal_channel_radius(x)

elif x > internal_channel_end:
    r = external_channel_radius(x)
else:
    external = random.choice([True, False])
    x_choices = [internal_channel_radius(x), external_channel_radius(x), ]
    r = x_choices[int(external)]

defect_theta = random.random() * 2 * np.pi

if external & (x < 33.5):
    r = random.choice([r - (1 - random.random()) * (defect_radius), (r - 1.75) - (1 - random.random()) * (defect_radius)])
else:
    r = r + (1 - random.random()) * (defect_radius * 2/3)

defect_y = r * np.cos(defect_theta)
defect_z = r * np.sin(defect_theta)

# Spherical Defect
if defect_type == 'sphere':
    if defect_morphology == 'void':
        nozzle = nozzle.cut(cq.Workplane("YZ").sphere(defect_radius).translate((-x, defect_y, defect_z)))
    else:
        nozzle = nozzle.union(cq.Workplane("YZ").sphere(defect_radius).translate((-x, defect_y, defect_z)))
# Cylinderical Defect
if defect_type == 'cylinder':
    if defect_morphology == 'void':
        nozzle = nozzle.cut(cq.Workplane("YZ").cylinder(defect_radius*1.5, defect_radius*.5).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
    else:
        nozzle = nozzle.union(cq.Workplane("YZ").cylinder(defect_radius*1.5, defect_radius*.5).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
# Rectangular Defect
if defect_type == 'rect':
    if defect_morphology == 'void':
        nozzle = nozzle.cut(cq.Workplane("YZ").rect(defect_radius*1.5, defect_radius*.5).extrude(random.random()*1.5*defect_radius).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
    else:
        nozzle = nozzle.union(cq.Workplane("YZ").rect(defect_radius*1.5, defect_radius*.5).extrude(random.random()*1.5*defect_radius).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
# Cone Defect
if defect_type == 'cone':
    if defect_morphology == 'void':
        nozzle = nozzle.cut(cq.Workplane("YZ").circle(defect_radius/2).workplane(offset=defect_radius*1.5).circle(0.01).loft(combine=True).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
    else:
        nozzle = nozzle.union(cq.Workplane("YZ").circle(defect_radius/2).workplane(offset=defect_radius*1.5).circle(0.01).loft(combine=True).translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
# spikey Defect
if defect_type == 'spikey':
    if defect_morphology == 'void':
        nozzle = nozzle.cut(spikey.translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
    else:
        nozzle = nozzle.union(spikey.translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
# blob Defect:
if defect_type == 'blob':
    if defect_morphology == 'void':
        nozzle = nozzle.cut(blob.translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))
    else:
        nozzle = nozzle.union(blob.translate((-x, defect_y, defect_z)).rotateAboutCenter((random.random(), random.random(), random.random()), random.random() * 360))

#exporters.export(nozzle, 'C:/Users/bimrose2/Box/ZJU UIUC - CT Defect Detection/Nozzle CAD/Synthetic_Defects/nozzle_r1_clog_' + str(id) + '.stl')
#nozzle.val().exportStl('C:/Users/bimrose2/Box/ZJU UIUC - CT Defect Detection/Nozzle CAD/Synthetic_Defects/nozzle_' + str(defect_type) + '_' + str(defect_morphology) + '_' + str(id) + '.stl',tolerance=0.001,angularTolerance=.2, ascii=False)

nozzle

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='dark', title=None, …

In [112]:
nozzle.val().exportStl('cad/nozzle_' + str(defect_type) + '_' + str(defect_morphology) + '_1mm.stl',tolerance=0.001,angularTolerance=.2, ascii=False)

True

In [27]:
def generate_sphere(size):
    blob = (cq.Workplane()
        .sphere(random.random()*size/3).translate(((random.random() - 1)*size, (random.random() - 1)*size, (random.random() - 1)*size))
        )
    return blob

def generate_blob(size, blob_num):
    blob = generate_sphere(size)
    for idx in range(blob_num):
        blob = blob.union(generate_sphere(size))
    return blob

blob = generate_blob(10, 100)
blob

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.99s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='dark', title=None, …

In [57]:
spikey = (
        cq.importers.importStep('cad/Great_stellated_dodecahedron.STEP') #path to dodecahedron
    )

In [86]:
import cadquery
import noise

def organic_blob(x, y, z):
    return noise.pnoise3(x, y, z, octaves=6, persistence=0.5, lacunarity=2)

def random_blob():
    b = cadquery.Workplane("XY").box(20, 20, 0.1).vertices()
    s = cadquery.Workplane("XY")
    s = s.pushPoints([(x, y, organic_blob(x, y, 0.5)) for x, y, z in b.points])
    return s.extrude(10)

result = random_blob()
result

AttributeError: 'Workplane' object has no attribute 'points'

In [87]:
def random_blob(size):
    s = (
         cq.Sketch()
        .arc((random.uniform(-size, size), random.uniform(-size, size)),size/2 + random.random()*size, 0.,360., tag='base')
        .arc((random.uniform(-size, size), random.uniform(-size, size)),size/2 + random.random()*size, 0.,360., tag='base')
        .arc((random.uniform(-size, size), random.uniform(-size, size)),size/2 + random.random()*size, 0.,360., tag='base')
        .arc((random.uniform(-size, size), random.uniform(-size, size)),size/2 + random.random()*size, 0.,360., tag='base')
        .hull(tag='base')
        # .polygon(((0, -10*size), (0, 10*size), (10*size, 10*size,), (10*size, -10*size), (0, -10*size)), mode='s', tag='base')
        )
    
    result = (
        cq.Workplane()
        .workplane(offset=0)
        .placeSketch(s)
        # .revolve(90)
        .extrude(2*size/3)
        .faces("|Z").edges().fillet(size/4)
    )

    return result

def generate_random_blob(size):
    result = None
    while result is None:
        try:
            # connect
            result = random_blob(size)
        except:
            pass
    return result

defect_size = 5

result = generate_random_blob(defect_size)



# result = (
#     cq.Workplane()
#     .placeSketch(generate_random_blob(defect_size), generate_random_blob(defect_size).moved(cq.Location(cq.Vector(0, 0, defect_size*3))))
#     .loft()
#     )

result

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='dark', title=None, …

In [88]:
def random_blob(size):
    s = (
         cq.Workplane()
        .sphere((random.uniform(-size, size), random.uniform(-size, size)),size/2 + random.random()*size, 0.,360., tag='base')
        .hull(tag='base')
        # .polygon(((0, -10*size), (0, 10*size), (10*size, 10*size,), (10*size, -10*size), (0, -10*size)), mode='s', tag='base')
        )
    
    result = (
        cq.Workplane()
        .workplane(offset=0)
        .placeSketch(s)
        # .revolve(90)
        .extrude(2*size/3)
        .faces("|Z").edges().fillet(size/4)
    )

    return result

def generate_random_blob(size):
    result = None
    while result is None:
        try:
            # connect
            result = random_blob(size)
        except:
            pass
    return result

defect_size = 5

result = generate_random_blob(defect_size)



# result = (
#     cq.Workplane()
#     .placeSketch(generate_random_blob(defect_size), generate_random_blob(defect_size).moved(cq.Location(cq.Vector(0, 0, defect_size*3))))
#     .loft()
#     )

result

In [ ]:

clone_result = cq.Workplane()

repeats = 10

for idx in range(repeats):
    clone_result.add(result.rotate((0, 0, 0), (random.random(), random.random(), random.random()), idx*190/repeats))

clone_result